# IMPORTING REQUIRED LIBRARIES AND CREATING REQUIRED FOLDERS FOR EFFICIENT PROGRAMMING

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import re
from tqdm import tqdm
import warnings

# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Embedding libraries
import gensim
from gensim.models import Word2Vec
from transformers import BertModel, BertTokenizer
import torch

# Dimensionality reduction
from sklearn.manifold import TSNE
from umap import UMAP

# Evaluation metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
warnings.filterwarnings('ignore')

# Create directories for outputs
os.makedirs('visualizations', exist_ok=True)
os.makedirs('reports', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

print("Setup complete!")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


Setup complete!


In [4]:
def load_and_examine_dataset(file_path, display_rows=5):
    """
    Load a dataset and display its basic information
    
    Args:
        file_path (str): Path to the dataset
        display_rows (int): Number of rows to display
        
    Returns:
        pd.DataFrame: Loaded dataset
    """
    print(f"Loading dataset: {file_path}")
    
    # Determine file extension and load accordingly
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, low_memory=False)
    elif file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_path}")
    
    print(f"\nDataset shape: {df.shape}")
    print("\nColumn information:")
    
    # Display column information
    col_info = []
    for col in df.columns:
        non_null = df[col].count()
        dtype = df[col].dtype
        unique = df[col].nunique()
        col_info.append({
            'Column': col, 
            'Non-Null Count': non_null,
            'Data Type': dtype,
            'Unique Values': unique,
            'Missing (%)': round(100 * (len(df) - non_null) / len(df), 2)
        })
    
    print(pd.DataFrame(col_info).to_string())
    
    print("\nSample data:")
    display_df = df.head(display_rows)
    
    return df

# Load all datasets
flood_df = load_and_examine_dataset('flood_data.csv')
shelter_df = load_and_examine_dataset('global_shelters_dataset.csv')
earthquake_df = load_and_examine_dataset('japan_earthquakes_1900-2024.csv')
landslide_df = load_and_examine_dataset('landslide_data.csv')

# Save basic dataset statistics
datasets = {
    'Floods': flood_df,
    'Shelters': shelter_df,
    'Earthquakes': earthquake_df,
    'Landslides': landslide_df
}

# Create a summary of all datasets
summary_data = []
for name, df in datasets.items():
    summary_data.append({
        'Dataset': name,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Missing Values (%)': round(100 * df.isna().sum().sum() / (df.shape[0] * df.shape[1]), 2)
    })

summary_df = pd.DataFrame(summary_data)
print("\nDatasets Summary:")
print(summary_df)

# Save the summary
summary_df.to_csv('reports/datasets_summary.csv', index=False)

Loading dataset: flood_data.csv

Dataset shape: (50000, 10)

Column information:
       Column  Non-Null Count Data Type  Unique Values  Missing (%)
0        time           50000    object          48788          0.0
1    latitude           50000   float64          50000          0.0
2   longitude           50000   float64          50000          0.0
3       depth           50000   float64          50000          0.0
4  waterLevel           50000   float64          50000          0.0
5       place           50000    object              5          0.0
6        type           50000    object              1          0.0
7  depthError           50000   float64          50000          0.0
8  levelError           50000   float64          50000          0.0
9          Id           50000     int64          50000          0.0

Sample data:
Loading dataset: global_shelters_dataset.csv

Dataset shape: (38775, 9)

Column information:
       Column  Non-Null Count Data Type  Unique Values  Missing 

# EDA TECHNIQUES

In [ ]:
def clean_text_data(text):
    """
    Clean text data by removing special characters and extra spaces
    
    Args:
        text (str): Text to clean
        
    Returns:
        str: Cleaned text
    """
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def preprocess_text(text):
    """
    Preprocess text data by tokenizing, removing stopwords, and lemmatizing
    
    Args:
        text (str): Text to preprocess
        
    Returns:
        list: List of preprocessed tokens
    """
    if not isinstance(text, str) or text == "":
        return []
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

def clean_numerical_data(df, column, bounds=None):
    """
    Clean numerical data by handling missing values and outliers
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        column (str): Column to clean
        bounds (tuple): Lower and upper bounds for valid values (optional)
        
    Returns:
        pd.Series: Cleaned numerical data
    """
    # Create a copy to avoid modifying the original
    cleaned = df[column].copy()
    
    # Handle missing values (replace with median)
    if cleaned.isna().any():
        median_value = cleaned.median()
        cleaned.fillna(median_value, inplace=True)
    
    # Handle outliers if bounds are provided
    if bounds:
        lower, upper = bounds
        cleaned = cleaned.clip(lower, upper)
    
    return cleaned

def clean_coordinate_data(df, lat_col, lon_col):
    """
    Clean coordinate data by handling invalid values
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        lat_col (str): Column name for latitude
        lon_col (str): Column name for longitude
        
    Returns:
        pd.DataFrame: DataFrame with cleaned coordinates
    """
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Define valid coordinate ranges
    lat_bounds = (-90, 90)
    lon_bounds = (-180, 180)
    
    # Handle invalid latitudes
    mask = (cleaned_df[lat_col] >= lat_bounds[0]) & (cleaned_df[lat_col] <= lat_bounds[1])
    cleaned_df.loc[~mask, lat_col] = np.nan
    
    # Handle invalid longitudes
    mask = (cleaned_df[lon_col] >= lon_bounds[0]) & (cleaned_df[lon_col] <= lon_bounds[1])
    cleaned_df.loc[~mask, lon_col] = np.nan
    
    return cleaned_df

# CLEANING DATA

In [6]:
def create_basic_visualizations(datasets):
    """Create basic visualizations for all datasets
    
    Args:
        datasets (dict): Dictionary of dataset name to DataFrame
    """
    # Set up the figure for all visualizations
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Dataset sizes comparison
    plt.subplot(2, 2, 1)
    sizes = [df.shape[0] for df in datasets.values()]
    plt.bar(datasets.keys(), sizes, color=sns.color_palette("viridis", len(datasets)))
    plt.title('Dataset Sizes (Number of Records)', fontsize=14)
    plt.xticks(rotation=45)
    plt.ylabel('Count')
    
    # 2. Missing values by dataset
    plt.subplot(2, 2, 2)
    missing_pcts = []
    for name, df in datasets.items():
        missing_pct = 100 * df.isna().sum().sum() / (df.shape[0] * df.shape[1])
        missing_pcts.append(missing_pct)
    
    plt.bar(datasets.keys(), missing_pcts, color=sns.color_palette("viridis", len(datasets)))
    plt.title('Missing Values Percentage by Dataset', fontsize=14)
    plt.xticks(rotation=45)
    plt.ylabel('Missing (%)')
    
    # 3. Shelter types distribution (if applicable)
    plt.subplot(2, 2, 3)
    if 'Shelters' in datasets and 'type' in datasets['Shelters'].columns:
        shelter_types = datasets['Shelters']['type'].value_counts()
        shelter_types.plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette("viridis", len(shelter_types)))
        plt.title('Shelter Types Distribution', fontsize=14)
        plt.ylabel('')
    
    # 4. Geographic distribution (basic world map with sample points)
    plt.subplot(2, 2, 4)
    
    # Create a basic world map showing sample data points if coordinates exist
    if 'Shelters' in datasets and 'latitude' in datasets['Shelters'].columns and 'longitude' in datasets['Shelters'].columns:
        sample_shelters = datasets['Shelters'][['latitude', 'longitude']].dropna().sample(min(500, len(datasets['Shelters'])))
        
        # Basic world map (very simplified)
        plt.scatter(sample_shelters['longitude'], sample_shelters['latitude'], 
                   alpha=0.5, s=10, c='blue', label='Shelters')
        plt.xlim(-180, 180)
        plt.ylim(-90, 90)
        plt.title('Sample Geographic Distribution', fontsize=14)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.grid(True, alpha=0.3)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('visualizations/dataset_overview.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Basic visualizations created and saved to 'visualizations/dataset_overview.png'")

# Create basic visualizations
create_basic_visualizations(datasets)

# Additional detailed explorations for each dataset
def explore_shelter_data(df):
    """Explore shelter dataset in more detail"""
    # Distribution of shelters by country
    plt.figure(figsize=(12, 6))
    country_counts = df['country'].value_counts().head(15)
    sns.barplot(x=country_counts.index, y=country_counts.values)
    plt.title('Top 15 Countries by Number of Shelters')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('visualizations/shelters_by_country.png', dpi=300)
    plt.close()
    
    # Distribution of shelters with known capacity
    plt.figure(figsize=(10, 6))
    capacity_data = df[df['capacity'].apply(lambda x: str(x).isdigit())]
    capacity_data['capacity'] = capacity_data['capacity'].astype(int)
    sns.histplot(capacity_data['capacity'], bins=20, kde=True)
    plt.title('Distribution of Shelter Capacity')
    plt.xlabel('Capacity')
    plt.savefig('visualizations/shelter_capacity_distribution.png', dpi=300)
    plt.close()
    
    # Distribution of shelter types
    if 'type' in df.columns:
        plt.figure(figsize=(12, 6))
        type_counts = df['type'].value_counts()
        sns.barplot(x=type_counts.index, y=type_counts.values)
        plt.title('Distribution of Shelter Types')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('visualizations/shelter_types_distribution.png', dpi=300)
        plt.close()
    
    return {
        'total_shelters': len(df),
        'countries_represented': df['country'].nunique(),
        'avg_capacity': capacity_data['capacity'].mean() if not capacity_data.empty else 0
    }

def explore_earthquake_data(df):
    """Explore earthquake dataset in more detail"""
    # Check necessary columns
    if not all(col in df.columns for col in ['magnitude']):
        return {'error': 'Required columns not found in earthquake dataset'}
    
    # Convert date column to datetime if exists
    date_cols = [col for col in df.columns if any(x in col.lower() for x in ['date', 'time'])]
    if date_cols:
        date_col = date_cols[0]
        try:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        except:
            pass
    
    # Distribution of earthquake magnitudes
    plt.figure(figsize=(10, 6))
    sns.histplot(df['magnitude'].dropna(), bins=20, kde=True)
    plt.title('Distribution of Earthquake Magnitudes')
    plt.xlabel('Magnitude')
    plt.savefig('visualizations/earthquake_magnitude_distribution.png', dpi=300)
    plt.close()
    
    # Earthquakes over time if date column exists
    if date_cols and not df[date_col].isna().all():
        plt.figure(figsize=(14, 6))
        
        # Group by year and count
        df_with_date = df.dropna(subset=[date_col])
        if 'year' not in df.columns and hasattr(df[date_col].dt, 'year'):
            df_with_date['year'] = df[date_col].dt.year
            yearly_counts = df_with_date.groupby('year').size()
            
            plt.plot(yearly_counts.index, yearly_counts.values)
            plt.title('Number of Earthquakes by Year')
            plt.xlabel('Year')
            plt.ylabel('Count')
            plt.grid(True, alpha=0.3)
            plt.savefig('visualizations/earthquakes_by_year.png', dpi=300)
            plt.close()
    
    # Distribution of earthquake depths if available
    if 'depth' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(df['depth'].dropna(), bins=20, kde=True)
        plt.title('Distribution of Earthquake Depths')
        plt.xlabel('Depth (km)')
        plt.savefig('visualizations/earthquake_depth_distribution.png', dpi=300)
        plt.close()
    
    # Geographic distribution of earthquakes
    if all(col in df.columns for col in ['latitude', 'longitude']):
        plt.figure(figsize=(12, 8))
        plt.scatter(df['longitude'], df['latitude'], alpha=0.5, 
                    s=df['magnitude']*2 if 'magnitude' in df.columns else 10, 
                    c=df['magnitude'] if 'magnitude' in df.columns else 'blue')
        plt.colorbar(label='Magnitude')
        plt.title('Geographic Distribution of Earthquakes')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.grid(True, alpha=0.3)
        plt.savefig('visualizations/earthquake_geographic_distribution.png', dpi=300)
        plt.close()
    
    return {
        'total_earthquakes': len(df),
        'avg_magnitude': df['magnitude'].mean(),
        'max_magnitude': df['magnitude'].max(),
        'min_magnitude': df['magnitude'].min()
    }

def explore_flood_data(df):
    """Explore flood dataset in more detail"""
    # Check columns to determine what visualizations are possible
    
    # Temporal analysis if date column exists
    date_cols = [col for col in df.columns if any(x in col.lower() for x in ['date', 'time'])]
    if date_cols:
        date_col = date_cols[0]
        try:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
            
            # Floods by month
            plt.figure(figsize=(12, 6))
            if hasattr(df[date_col].dt, 'month'):
                monthly_counts = df[date_col].dt.month.value_counts().sort_index()
                sns.barplot(x=monthly_counts.index, y=monthly_counts.values)
                plt.title('Flood Events by Month')
                plt.xlabel('Month')
                plt.ylabel('Count')
                plt.xticks(range(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
                plt.tight_layout()
                plt.savefig('visualizations/floods_by_month.png', dpi=300)
                plt.close()
        except:
            pass
    
    # Location analysis
    location_cols = [col for col in df.columns if any(x in col.lower() for x in ['country', 'location', 'region'])]
    if location_cols:
        location_col = location_cols[0]
        plt.figure(figsize=(14, 8))
        location_counts = df[location_col].value_counts().head(15)
        sns.barplot(x=location_counts.index, y=location_counts.values)
        plt.title('Top 15 Locations with Flood Events')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('visualizations/flood_locations.png', dpi=300)
        plt.close()
    
    # Severity analysis if available
    severity_cols = [col for col in df.columns if any(x in col.lower() for x in ['severity', 'impact', 'damage', 'affected'])]
    if severity_cols:
        severity_col = severity_cols[0]
        if pd.api.types.is_numeric_dtype(df[severity_col]):
            plt.figure(figsize=(10, 6))
            sns.histplot(df[severity_col].dropna(), bins=20, kde=True)
            plt.title(f'Distribution of Flood {severity_col.title()}')
            plt.xlabel(severity_col.title())
            plt.tight_layout()
            plt.savefig(f'visualizations/flood_{severity_col}_distribution.png', dpi=300)
            plt.close()
    
    # Geographic distribution if coordinates available
    if all(col in df.columns for col in ['latitude', 'longitude']):
        plt.figure(figsize=(12, 8))
        plt.scatter(df['longitude'], df['latitude'], alpha=0.5, s=10, c='blue')
        plt.title('Geographic Distribution of Flood Events')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.grid(True, alpha=0.3)
        plt.savefig('visualizations/flood_geographic_distribution.png', dpi=300)
        plt.close()
    
    return {
        'total_floods': len(df),
        'time_period': f"{df[date_cols[0]].min()} to {df[date_cols[0]].max()}" if date_cols else "Unknown"
    }

def explore_landslide_data(df):
    """Explore landslide dataset in more detail"""
    # Check columns to determine what visualizations are possible
    
    # Temporal analysis if date column exists
    date_cols = [col for col in df.columns if any(x in col.lower() for x in ['date', 'time'])]
    if date_cols:
        date_col = date_cols[0]
        try:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
            
            # Landslides over time
            plt.figure(figsize=(14, 6))
            if hasattr(df[date_col].dt, 'year'):
                df_with_date = df.dropna(subset=[date_col])
                df_with_date['year'] = df[date_col].dt.year
                yearly_counts = df_with_date.groupby('year').size()
                
                plt.plot(yearly_counts.index, yearly_counts.values)
                plt.title('Number of Landslide Events by Year')
                plt.xlabel('Year')
                plt.ylabel('Count')
                plt.grid(True, alpha=0.3)
                plt.savefig('visualizations/landslides_by_year.png', dpi=300)
                plt.close()
        except:
            pass
    
    # Trigger analysis if trigger column exists
    trigger_cols = [col for col in df.columns if any(x in col.lower() for x in ['trigger', 'cause'])]
    if trigger_cols:
        trigger_col = trigger_cols[0]
        plt.figure(figsize=(14, 8))
        trigger_counts = df[trigger_col].value_counts().head(10)
        sns.barplot(x=trigger_counts.index, y=trigger_counts.values)
        plt.title('Top 10 Landslide Triggers')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('visualizations/landslide_triggers.png', dpi=300)
        plt.close()
    
    # Size/category analysis if available
    size_cols = [col for col in df.columns if any(x in col.lower() for x in ['size', 'category', 'class'])]
    if size_cols:
        size_col = size_cols[0]
        plt.figure(figsize=(12, 6))
        size_counts = df[size_col].value_counts()
        sns.barplot(x=size_counts.index, y=size_counts.values)
        plt.title('Landslide Size/Category Distribution')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('visualizations/landslide_sizes.png', dpi=300)
        plt.close()
    
    # Fatality analysis if available
    fatality_cols = [col for col in df.columns if any(x in col.lower() for x in ['fatal', 'death', 'casualty'])]
    if fatality_cols:
        fatality_col = fatality_cols[0]
        if pd.api.types.is_numeric_dtype(df[fatality_col]):
            plt.figure(figsize=(10, 6))
            sns.histplot(df[df[fatality_col] > 0][fatality_col], bins=20, kde=True)
            plt.title('Distribution of Fatalities in Landslide Events')
            plt.xlabel('Number of Fatalities')
            plt.tight_layout()
            plt.savefig('visualizations/landslide_fatalities.png', dpi=300)
            plt.close()
    
    # Geographic distribution if coordinates available
    if all(col in df.columns for col in ['latitude', 'longitude']):
        plt.figure(figsize=(12, 8))
        plt.scatter(df['longitude'], df['latitude'], alpha=0.5, s=10, c='green')
        plt.title('Geographic Distribution of Landslide Events')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.grid(True, alpha=0.3)
        plt.savefig('visualizations/landslide_geographic_distribution.png', dpi=300)
        plt.close()
    
    return {
        'total_landslides': len(df),
        'time_period': f"{df[date_cols[0]].min()} to {df[date_cols[0]].max()}" if date_cols else "Unknown"
    }

# Run detailed exploration for all datasets
shelter_insights = explore_shelter_data(shelter_df)
print("Shelter Data Insights:", shelter_insights)

earthquake_insights = explore_earthquake_data(earthquake_df)
print("Earthquake Data Insights:", earthquake_insights)

flood_insights = explore_flood_data(flood_df)
print("Flood Data Insights:", flood_insights)

landslide_insights = explore_landslide_data(landslide_df)
print("Landslide Data Insights:", landslide_insights)

# Save all insights to a summary file
all_insights = {
    'Shelters': shelter_insights,
    'Earthquakes': earthquake_insights,
    'Floods': flood_insights,
    'Landslides': landslide_insights
}

# Convert to DataFrame for easy saving
insights_df = pd.DataFrame({
    'Dataset': list(all_insights.keys()),
    'Key Metrics': [str(insight) for insight in all_insights.values()]
})

# Save insights summary
insights_df.to_csv('reports/dataset_insights_summary.csv', index=False)
print("All dataset insights saved to 'reports/dataset_insights_summary.csv'")

Basic visualizations created and saved to 'visualizations/dataset_overview.png'
Shelter Data Insights: {'total_shelters': 38775, 'countries_represented': 3, 'avg_capacity': 7.411764705882353}
Earthquake Data Insights: {'error': 'Required columns not found in earthquake dataset'}
Flood Data Insights: {'total_floods': 50000, 'time_period': '2015-01-01 00:29:00 to 2016-11-25 10:23:00'}
Landslide Data Insights: {'total_landslides': 50000, 'time_period': '2010-01-01 01:24:00 to 2011-11-26 10:24:00'}
All dataset insights saved to 'reports/dataset_insights_summary.csv'


# APPLYING CLEANING FUNCTIONS TO DATASETS

In [9]:
def clean_text_data(text):
    """
    Clean text data by removing special characters and extra spaces
    
    Args:
        text (str): Text to clean
        
    Returns:
        str: Cleaned text
    """
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def preprocess_text(text):
    """
    Preprocess text data by tokenizing, removing stopwords, and lemmatizing
    
    Args:
        text (str): Text to preprocess
        
    Returns:
        list: List of preprocessed tokens
    """
    if not isinstance(text, str) or text == "":
        return []
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

def clean_numerical_data(df, column, bounds=None):
    """
    Clean numerical data by handling missing values and outliers
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        column (str): Column to clean
        bounds (tuple): Lower and upper bounds for valid values (optional)
        
    Returns:
        pd.Series: Cleaned numerical data
    """
    # Create a copy to avoid modifying the original
    cleaned = df[column].copy()
    
    # Handle missing values (replace with median)
    if cleaned.isna().any():
        median_value = cleaned.median()
        cleaned.fillna(median_value, inplace=True)
    
    # Handle outliers if bounds are provided
    if bounds:
        lower, upper = bounds
        cleaned = cleaned.clip(lower, upper)
    
    return cleaned

def clean_coordinate_data(df, lat_col, lon_col):
    """
    Clean coordinate data by handling invalid values
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        lat_col (str): Column name for latitude
        lon_col (str): Column name for longitude
        
    Returns:
        pd.DataFrame: DataFrame with cleaned coordinates
    """
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Define valid coordinate ranges
    lat_bounds = (-90, 90)
    lon_bounds = (-180, 180)
    
    # Handle invalid latitudes
    mask = (cleaned_df[lat_col] >= lat_bounds[0]) & (cleaned_df[lat_col] <= lat_bounds[1])
    cleaned_df.loc[~mask, lat_col] = np.nan
    
    # Handle invalid longitudes
    mask = (cleaned_df[lon_col] >= lon_bounds[0]) & (cleaned_df[lon_col] <= lon_bounds[1])
    cleaned_df.loc[~mask, lon_col] = np.nan
    
    return cleaned_df

# Apply Cleaning Functions to Data Sets


In [11]:
def clean_shelter_data(df):
    """
    Clean the shelter dataset
    
    Args:
        df (pd.DataFrame): Shelter dataset
        
    Returns:
        pd.DataFrame: Cleaned shelter dataset
    """
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Clean coordinate data
    cleaned_df = clean_coordinate_data(cleaned_df, 'latitude', 'longitude')
    
    # Clean text data
    if 'name' in cleaned_df.columns:
        cleaned_df['name_cleaned'] = cleaned_df['name'].apply(clean_text_data)
    
    # Handle capacity data
    if 'capacity' in cleaned_df.columns:
        # Convert to numeric, forcing errors to NaN
        cleaned_df['capacity'] = pd.to_numeric(cleaned_df['capacity'], errors='coerce')
        
        # Replace NaN with 0 (unknown capacity)
        cleaned_df['capacity'].fillna(0, inplace=True)
    
    # Clean country data (standardize country names)
    if 'country' in cleaned_df.columns:
        cleaned_df['country'] = cleaned_df['country'].str.strip()
        
        # Handle missing countries
        cleaned_df['country'].fillna('Unknown', inplace=True)
    
    # Clean type data
    if 'type' in cleaned_df.columns:
        cleaned_df['type'] = cleaned_df['type'].str.strip().str.lower()
        cleaned_df['type'].fillna('unknown', inplace=True)
    
    return cleaned_df

def clean_earthquake_data(df):
    """
    Clean the earthquake dataset
    
    Args:
        df (pd.DataFrame): Earthquake dataset
        
    Returns:
        pd.DataFrame: Cleaned earthquake dataset
    """
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Handle magnitude data (assuming 'magnitude' column exists)
    if 'magnitude' in cleaned_df.columns:
        # Convert to numeric, forcing errors to NaN
        cleaned_df['magnitude'] = pd.to_numeric(cleaned_df['magnitude'], errors='coerce')
        
        # Replace NaN with median
        median_magnitude = cleaned_df['magnitude'].median()
        cleaned_df['magnitude'].fillna(median_magnitude, inplace=True)
        
        # Handle outliers (clip to reasonable range)
        cleaned_df['magnitude'] = cleaned_df['magnitude'].clip(0, 10)
    
    # Clean date/time data (assuming 'date' or 'time' column exists)
    date_cols = [col for col in cleaned_df.columns if any(x in col.lower() for x in ['date', 'time'])]
    for col in date_cols:
        try:
            cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce')
        except:
            # If conversion fails, keep as is
            pass
    
    # Clean coordinate data (assuming latitude and longitude columns exist)
    lat_col = next((col for col in cleaned_df.columns if 'lat' in col.lower()), None)
    lon_col = next((col for col in cleaned_df.columns if 'lon' in col.lower()), None)
    
    if lat_col and lon_col:
        cleaned_df = clean_coordinate_data(cleaned_df, lat_col, lon_col)
    
    return cleaned_df

def clean_flood_data(df):
    """
    Clean the flood dataset
    
    Args:
        df (pd.DataFrame): Flood dataset
        
    Returns:
        pd.DataFrame: Cleaned flood dataset
    """
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Clean text fields (descriptions, locations, etc.)
    text_cols = [col for col in cleaned_df.columns if cleaned_df[col].dtype == 'object']
    for col in text_cols:
        cleaned_df[f'{col}_cleaned'] = cleaned_df[col].apply(clean_text_data)
    
    # Handle date/time data
    date_cols = [col for col in cleaned_df.columns if any(x in col.lower() for x in ['date', 'time'])]
    for col in date_cols:
        try:
            cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce')
        except:
            # If conversion fails, keep as is
            pass
    
    # Clean coordinate data
    lat_col = next((col for col in cleaned_df.columns if 'lat' in col.lower()), None)
    lon_col = next((col for col in cleaned_df.columns if 'lon' in col.lower()), None)
    
    if lat_col and lon_col:
        cleaned_df = clean_coordinate_data(cleaned_df, lat_col, lon_col)
    
    return cleaned_df

def clean_landslide_data(df):
    """
    Clean the landslide dataset
    
    Args:
        df (pd.DataFrame): Landslide dataset
        
    Returns:
        pd.DataFrame: Cleaned landslide dataset
    """
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Clean text fields (descriptions, locations, etc.)
    text_cols = [col for col in cleaned_df.columns if cleaned_df[col].dtype == 'object']
    for col in text_cols:
        cleaned_df[f'{col}_cleaned'] = cleaned_df[col].apply(clean_text_data)
    
    # Handle date/time data
    date_cols = [col for col in cleaned_df.columns if any(x in col.lower() for x in ['date', 'time'])]
    for col in date_cols:
        try:
            cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce')
        except:
            # If conversion fails, keep as is
            pass
    
    # Clean coordinate data
    lat_col = next((col for col in cleaned_df.columns if 'lat' in col.lower()), None)
    lon_col = next((col for col in cleaned_df.columns if 'lon' in col.lower()), None)
    
    if lat_col and lon_col:
        cleaned_df = clean_coordinate_data(cleaned_df, lat_col, lon_col)
    
    return cleaned_df

# Apply cleaning functions to datasets
cleaned_shelter_df = clean_shelter_data(shelter_df)
cleaned_earthquake_df = clean_earthquake_data(earthquake_df)
cleaned_flood_df = clean_flood_data(flood_df)
cleaned_landslide_df = clean_landslide_data(landslide_df)

# Store cleaned datasets in a dictionary
cleaned_datasets = {
    'Shelters': cleaned_shelter_df,
    'Earthquakes': cleaned_earthquake_df,
    'Floods': cleaned_flood_df,
    'Landslides': cleaned_landslide_df
}

# Save cleaned datasets
for name, df in cleaned_datasets.items():
    df.to_csv(f'data/cleaned_{name.lower()}.csv', index=False)
    print(f"Saved cleaned {name} dataset with shape {df.shape}")

# Display cleaning summary
cleaning_summary = []
for name in datasets.keys():
    orig_df = datasets[name]
    cleaned_df = cleaned_datasets[name]
    
    # Calculate missing values before and after cleaning
    orig_missing = 100 * orig_df.isna().sum().sum() / (orig_df.shape[0] * orig_df.shape[1])
    cleaned_missing = 100 * cleaned_df.isna().sum().sum() / (cleaned_df.shape[0] * cleaned_df.shape[1])
    
    cleaning_summary.append({
        'Dataset': name,
        'Original Shape': orig_df.shape,
        'Cleaned Shape': cleaned_df.shape,
        'Original Missing (%)': round(orig_missing, 2),
        'Cleaned Missing (%)': round(cleaned_missing, 2),
        'Improvement (%)': round(orig_missing - cleaned_missing, 2)
    })

# Display cleaning summary
cleaning_summary_df = pd.DataFrame(cleaning_summary)
print("\nData Cleaning Summary:")
print(cleaning_summary_df)

# Save cleaning summary
cleaning_summary_df.to_csv('reports/cleaning_summary.csv', index=False)

# Create visualization of cleaning improvements
plt.figure(figsize=(12, 6))
datasets = cleaning_summary_df['Dataset']
orig_missing = cleaning_summary_df['Original Missing (%)']
cleaned_missing = cleaning_summary_df['Cleaned Missing (%)']

x = np.arange(len(datasets))
width = 0.35

plt.bar(x - width/2, orig_missing, width, label='Original')
plt.bar(x + width/2, cleaned_missing, width, label='Cleaned')

plt.xlabel('Dataset')
plt.ylabel('Missing Values (%)')
plt.title('Improvement in Data Quality After Cleaning')
plt.xticks(x, datasets)
plt.legend()

plt.tight_layout()
plt.savefig('visualizations/cleaning_improvement.png', dpi=300)
plt.close()

Saved cleaned Shelters dataset with shape (38775, 10)
Saved cleaned Earthquakes dataset with shape (51275, 22)
Saved cleaned Floods dataset with shape (50000, 12)
Saved cleaned Landslides dataset with shape (50000, 12)

Data Cleaning Summary:
       Dataset Original Shape Cleaned Shape  Original Missing (%)  \
0       Floods    (50000, 10)   (50000, 12)                  0.00   
1     Shelters     (38775, 9)   (38775, 10)                 10.76   
2  Earthquakes    (51275, 22)   (51275, 22)                 18.31   
3   Landslides    (50000, 10)   (50000, 12)                  0.00   

   Cleaned Missing (%)  Improvement (%)  
0                 0.00             0.00  
1                 9.68             1.08  
2                18.31             0.00  
3                 0.00             0.00  


# Feature Engineering Functions

In [12]:
def extract_temporal_features(df, date_col):
    """
    Extract temporal features from a date/time column
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        date_col (str): Column name containing datetime data
        
    Returns:
        pd.DataFrame: DataFrame with new temporal features
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Ensure the date column is datetime type
    if pd.api.types.is_datetime64_any_dtype(result_df[date_col]):
        # Extract temporal features
        result_df[f'{date_col}_year'] = result_df[date_col].dt.year
        result_df[f'{date_col}_month'] = result_df[date_col].dt.month
        result_df[f'{date_col}_day'] = result_df[date_col].dt.day
        result_df[f'{date_col}_hour'] = result_df[date_col].dt.hour
        result_df[f'{date_col}_dayofweek'] = result_df[date_col].dt.dayofweek
        result_df[f'{date_col}_quarter'] = result_df[date_col].dt.quarter
        
        # Create cyclical features for time components
        # This helps with the circular nature of time (e.g., December is close to January)
        result_df[f'{date_col}_month_sin'] = np.sin(2 * np.pi * result_df[f'{date_col}_month'] / 12)
        result_df[f'{date_col}_month_cos'] = np.cos(2 * np.pi * result_df[f'{date_col}_month'] / 12)
        result_df[f'{date_col}_hour_sin'] = np.sin(2 * np.pi * result_df[f'{date_col}_hour'] / 24)
        result_df[f'{date_col}_hour_cos'] = np.cos(2 * np.pi * result_df[f'{date_col}_hour'] / 24)
        result_df[f'{date_col}_day_sin'] = np.sin(2 * np.pi * result_df[f'{date_col}_day'] / 31)
        result_df[f'{date_col}_day_cos'] = np.cos(2 * np.pi * result_df[f'{date_col}_day'] / 31)
        
        # Is the event during a weekend?
        result_df[f'{date_col}_is_weekend'] = result_df[f'{date_col}_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)
        
        # Is the event during night hours? (8pm-6am)
        result_df[f'{date_col}_is_night'] = result_df[f'{date_col}_hour'].apply(lambda x: 1 if x >= 20 or x < 6 else 0)
        
        print(f"Added temporal features for {date_col}")
    else:
        print(f"Column {date_col} is not a datetime column")
    
    return result_df

def engineer_geospatial_features(df, lat_col, lon_col):
    """
    Engineer geospatial features from latitude and longitude data
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        lat_col (str): Column name for latitude
        lon_col (str): Column name for longitude
        
    Returns:
        pd.DataFrame: DataFrame with new geospatial features
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Calculate distance from the equator (absolute latitude)
    result_df['distance_from_equator'] = result_df[lat_col].abs()
    
    # Calculate distance from the prime meridian (absolute longitude)
    result_df['distance_from_meridian'] = result_df[lon_col].abs()
    
    # Determine hemisphere (North/South, East/West)
    result_df['northern_hemisphere'] = (result_df[lat_col] >= 0).astype(int)
    result_df['eastern_hemisphere'] = (result_df[lon_col] >= 0).astype(int)
    
    # Simplified continent categorization based on rough coordinates
    def get_simplified_region(lat, lon):
        if (lat > 30 and lat < 75) and (lon > -25 and lon < 40):
            return 'Europe'
        elif (lat > 20 and lat < 80) and (lon > 30 and lon < 180):
            return 'Asia'
        elif (lat > -60 and lat < 30) and (lon > -20 and lon < 60):
            return 'Africa'
        elif (lat > 15 and lat < 85) and (lon > -170 and lon < -50):
            return 'North America'
        elif (lat > -60 and lat < 15) and (lon > -90 and lon < -30):
            return 'South America'
        elif (lat > -50 and lat < 0) and (lon > 110 and lon < 180):
            return 'Oceania'
        else:
            return 'Other'
    
    # Apply the simplified region categorization
    result_df['simplified_region'] = result_df.apply(
        lambda row: get_simplified_region(row[lat_col], row[lon_col]), axis=1
    )
    
    # Check if the location is coastal (within 1 degree of a coastline)
    # This is a simplified approximation - a real implementation would use a coastline dataset
    # For now, we'll just mark it as a placeholder feature
    result_df['is_coastal'] = 0
    
    print(f"Added geospatial features using {lat_col} and {lon_col}")
    
    return result_df

def normalize_numerical_features(df, exclude_cols=None):
    """
    Normalize numerical features in the DataFrame
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        exclude_cols (list): List of columns to exclude from normalization
        
    Returns:
        pd.DataFrame: DataFrame with normalized numerical features
        dict: Scaler objects for each column
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    if exclude_cols is None:
        exclude_cols = []
    
    # Dictionary to store scalers
    scalers = {}
    
    # Identify numerical columns
    numerical_cols = [col for col in result_df.columns 
                    if pd.api.types.is_numeric_dtype(result_df[col]) 
                    and col not in exclude_cols]
    
    # Normalize each column independently
    for col in numerical_cols:
        # Get min and max values
        min_val = result_df[col].min()
        max_val = result_df[col].max()
        
        # Store scaler parameters
        scalers[col] = {'min': min_val, 'max': max_val}
        
        # Normalize the column (min-max scaling)
        if max_val > min_val:
            result_df[col] = (result_df[col] - min_val) / (max_val - min_val)
        else:
            # Handle constant columns
            result_df[col] = 0.5
    
    print(f"Normalized {len(numerical_cols)} numerical columns")
    
    return result_df, scalers

def engineer_text_features(df, text_col, max_features=100):
    """
    Engineer features from text data using TF-IDF
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        text_col (str): Column name containing text data
        max_features (int): Maximum number of features to extract
        
    Returns:
        pd.DataFrame: DataFrame with new text features
        TfidfVectorizer: Fitted vectorizer
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Make sure all values are strings
    result_df[text_col] = result_df[text_col].fillna('').astype(str)
    
    # Create and fit TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(
        max_features=max_features,
        stop_words='english',
        ngram_range=(1, 2)
    )
    
    # Transform text to TF-IDF features
    tfidf_features = tfidf_vectorizer.fit_transform(result_df[text_col])
    
    # Convert to DataFrame
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=[f'tfidf_{text_col}_{f}' for f in feature_names])
    
    # Concatenate with original DataFrame
    result_df = pd.concat([result_df, tfidf_df], axis=1)
    
    print(f"Added {tfidf_df.shape[1]} TF-IDF features from {text_col}")
    
    return result_df, tfidf_vectorizer

def create_disaster_specific_features(df, disaster_type):
    """
    Create disaster-specific features based on the type of disaster
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        disaster_type (str): Type of disaster ('flood', 'earthquake', 'landslide', 'shelter')
        
    Returns:
        pd.DataFrame: DataFrame with new disaster-specific features
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    if disaster_type.lower() == 'flood':
        # Add water-related features
        if 'distance_from_equator' in result_df.columns:
            # Monsoon likelihood (higher near the equator)
            result_df['monsoon_likelihood'] = 1 - (result_df['distance_from_equator'] / 90)
        
        # Seasonality (northern hemisphere has more floods in spring/summer)
        if any(col.endswith('_month') for col in result_df.columns):
            month_col = next(col for col in result_df.columns if col.endswith('_month'))
            
            # Northern hemisphere flood season (roughly April-August, months 4-8)
            northern_flood_season = result_df.apply(
                lambda row: 1 if row['northern_hemisphere'] == 1 and 4 <= row[month_col] <= 8 else 0, 
                axis=1
            )
            
            # Southern hemisphere flood season (roughly October-February, months 10-2)
            southern_flood_season = result_df.apply(
                lambda row: 1 if row['northern_hemisphere'] == 0 and (row[month_col] >= 10 or row[month_col] <= 2) else 0,
                axis=1
            )
            
            result_df['in_flood_season'] = northern_flood_season | southern_flood_season
    
    elif disaster_type.lower() == 'earthquake':
        # For earthquakes, add features related to magnitude and depth
        if 'magnitude' in result_df.columns:
            # Categorize earthquake magnitude
            result_df['magnitude_category'] = pd.cut(
                result_df['magnitude'], 
                bins=[0, 2, 4, 6, 8, 10], 
                labels=['Very minor', 'Minor', 'Moderate', 'Strong', 'Major']
            )
            
            # Damage potential (simplified estimation based on magnitude)
            result_df['damage_potential'] = result_df['magnitude'].apply(lambda x: min(1.0, (x ** 2) / 80))
        
        # Depth-related features
        if 'depth' in result_df.columns:
            # Categorize earthquake depth
            result_df['depth_category'] = pd.cut(
                result_df['depth'], 
                bins=[0, 30, 70, 300, 700], 
                labels=['Shallow', 'Intermediate', 'Deep', 'Very deep']
            )
            
            # Surface impact (inverse relationship with depth)
            result_df['surface_impact'] = 1 / (1 + result_df['depth'] / 50)
    
    elif disaster_type.lower() == 'landslide':
        # For landslides, we can add terrain-related features
        if 'distance_from_equator' in result_df.columns:
            # Mountain region likelihood (simplified estimation)
            # Major mountain ranges tend to be at specific latitudes
            result_df['mountain_zone_likelihood'] = result_df['distance_from_equator'].apply(
                lambda x: 0.8 if (x > 25 and x < 45) else (0.6 if (x > 15 and x < 55) else 0.3)
            )
        
        # Seasonality (rainy seasons increase landslide risk)
        if any(col.endswith('_month') for col in result_df.columns):
            month_col = next(col for col in result_df.columns if col.endswith('_month'))
            
            # Very simplified rainy season indicator
            northern_rainy = result_df.apply(
                lambda row: 1 if row['northern_hemisphere'] == 1 and 6 <= row[month_col] <= 9 else 0, 
                axis=1
            )
            
            southern_rainy = result_df.apply(
                lambda row: 1 if row['northern_hemisphere'] == 0 and (row[month_col] >= 12 or row[month_col] <= 3) else 0,
                axis=1
            )
            
            result_df['in_rainy_season'] = northern_rainy | southern_rainy
    
    elif disaster_type.lower() == 'shelter':
        # For shelters, we can add capacity and location-based features
        if 'capacity' in result_df.columns:
            # Categorize shelter capacity
            result_df['capacity_category'] = pd.cut(
                result_df['capacity'], 
                bins=[0, 50, 100, 500, 1000, float('inf')], 
                labels=['Tiny', 'Small', 'Medium', 'Large', 'Very large']
            )
        
        # Add urban/rural estimation if we have coordinates
        if 'latitude' in result_df.columns and 'longitude' in result_df.columns:
            # This is a very simplified proxy using coordinate precision as an estimate
            # (In real applications, you'd use population density data)
            result_df['urban_likelihood'] = 0.5
    
    print(f"Added disaster-specific features for {disaster_type}")
    
    return result_df

# Apply Feature Engineering to Each Dataset

In [13]:
def run_feature_engineering(df, disaster_type, include_text=True, text_col=None):
    """
    Run complete feature engineering pipeline on a dataset
    
    Args:
        df (pd.DataFrame): DataFrame to process
        disaster_type (str): Type of disaster
        include_text (bool): Whether to include text feature engineering
        text_col (str): Column name containing text to process
        
    Returns:
        pd.DataFrame: Processed DataFrame with engineered features
    """
    print(f"Starting feature engineering for {disaster_type} dataset")
    result_df = df.copy()
    
    # 1. Extract temporal features from date columns
    date_cols = [col for col in result_df.columns if pd.api.types.is_datetime64_any_dtype(result_df[col])]
    for date_col in date_cols:
        result_df = extract_temporal_features(result_df, date_col)
    
    # 2. Engineer geospatial features if lat/lon columns exist
    lat_col = next((col for col in result_df.columns if 'lat' in col.lower()), None)
    lon_col = next((col for col in result_df.columns if 'lon' in col.lower()), None)
    
    if lat_col and lon_col:
        result_df = engineer_geospatial_features(result_df, lat_col, lon_col)
    
    # 3. Create disaster-specific features
    result_df = create_disaster_specific_features(result_df, disaster_type)
    
    # 4. Engineer text features if requested and text column exists
    vectorizer = None
    if include_text and text_col is not None and text_col in result_df.columns:
        result_df, vectorizer = engineer_text_features(result_df, text_col)
    
    # 5. Normalize numerical features
    # Exclude categorical and ID columns
    exclude_cols = date_cols.copy()
    if lat_col and lon_col:
        exclude_cols.extend([lat_col, lon_col])
    
    # Add any categorical columns to exclude list
    for col in result_df.columns:
        if pd.api.types.is_categorical_dtype(result_df[col]):
            exclude_cols.append(col)
    
    result_df, scalers = normalize_numerical_features(result_df, exclude_cols)
    
    print(f"Feature engineering complete for {disaster_type} dataset")
    print(f"Original shape: {df.shape}, New shape: {result_df.shape}")
    
    return result_df, vectorizer, scalers

# Apply feature engineering to each dataset
fe_flood_df, flood_vectorizer, flood_scalers = run_feature_engineering(
    cleaned_flood_df, 'flood', include_text=True, text_col='location'
)

fe_earthquake_df, earthquake_vectorizer, earthquake_scalers = run_feature_engineering(
    cleaned_earthquake_df, 'earthquake', include_text=False
)

fe_landslide_df, landslide_vectorizer, landslide_scalers = run_feature_engineering(
    cleaned_landslide_df, 'landslide', include_text=True, 
    text_col=next((col for col in cleaned_landslide_df.columns if 'trigger' in col.lower()), None)
)

fe_shelter_df, shelter_vectorizer, shelter_scalers = run_feature_engineering(
    cleaned_shelter_df, 'shelter', include_text=True, text_col='name'
)

# Store feature-engineered datasets
feature_engineered_datasets = {
    'Floods': fe_flood_df,
    'Earthquakes': fe_earthquake_df,
    'Landslides': fe_landslide_df,
    'Shelters': fe_shelter_df
}

# Save feature-engineered datasets
for name, df in feature_engineered_datasets.items():
    df.to_csv(f'data/fe_{name.lower()}.csv', index=False)
    print(f"Saved feature-engineered {name} dataset with shape {df.shape}")

# Recreate the datasets dictionary
datasets = {
    'Floods': flood_df,
    'Shelters': shelter_df,
    'Earthquakes': earthquake_df,
    'Landslides': landslide_df
}

# Create a summary of the feature engineering
fe_summary = []
for name in ['Floods', 'Earthquakes', 'Landslides', 'Shelters']:
    orig_df = datasets[name]
    cleaned_df = cleaned_datasets[name]
    fe_df = feature_engineered_datasets[name]
    
    fe_summary.append({
        'Dataset': name,
        'Original Shape': orig_df.shape,
        'Cleaned Shape': cleaned_df.shape,
        'Feature-Engineered Shape': fe_df.shape,
        'New Features Added': fe_df.shape[1] - cleaned_df.shape[1]
    })

# Display feature engineering summary
fe_summary_df = pd.DataFrame(fe_summary)
print("\nFeature Engineering Summary:")
print(fe_summary_df)

# Save feature engineering summary
fe_summary_df.to_csv('reports/feature_engineering_summary.csv', index=False)

# Create visualization of feature growth
plt.figure(figsize=(12, 6))
datasets = fe_summary_df['Dataset']
orig_features = [shape[1] for shape in fe_summary_df['Original Shape']]
cleaned_features = [shape[1] for shape in fe_summary_df['Cleaned Shape']]
fe_features = [shape[1] for shape in fe_summary_df['Feature-Engineered Shape']]

x = np.arange(len(datasets))
width = 0.25

plt.bar(x - width, orig_features, width, label='Original')
plt.bar(x, cleaned_features, width, label='Cleaned')
plt.bar(x + width, fe_features, width, label='Feature-Engineered')

plt.xlabel('Dataset')
plt.ylabel('Number of Features')
plt.title('Growth in Number of Features After Engineering')
plt.xticks(x, datasets)
plt.legend()

plt.tight_layout()
plt.savefig('visualizations/feature_growth.png', dpi=300)
plt.close()

Starting feature engineering for flood dataset
Added temporal features for time
Added geospatial features using latitude and longitude
Added disaster-specific features for flood
Normalized 26 numerical columns
Feature engineering complete for flood dataset
Original shape: (50000, 12), New shape: (50000, 34)
Starting feature engineering for earthquake dataset
Added temporal features for time
Added temporal features for updated
Added geospatial features using latitude and longitude
Added disaster-specific features for earthquake
Normalized 44 numerical columns
Feature engineering complete for earthquake dataset
Original shape: (51275, 22), New shape: (51275, 58)
Starting feature engineering for landslide dataset
Added temporal features for time
Added geospatial features using latitude and longitude
Added disaster-specific features for landslide
Normalized 26 numerical columns
Feature engineering complete for landslide dataset
Original shape: (50000, 12), New shape: (50000, 34)
Starting f

# Creating Embedding for DataSets

In [14]:
def prepare_data_for_embedding(df, text_cols=None, exclude_cols=None):
    """
    Prepare data for embedding by selecting appropriate columns and formatting
    
    Args:
        df (pd.DataFrame): DataFrame to process
        text_cols (list): List of text columns to include
        exclude_cols (list): List of columns to exclude
        
    Returns:
        pd.DataFrame: DataFrame ready for embedding
    """
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    if exclude_cols is None:
        exclude_cols = []
    
    if text_cols is None:
        text_cols = []
    
    # Keep only numeric columns and specified text columns
    keep_cols = []
    for col in result_df.columns:
        # Keep numeric columns
        if pd.api.types.is_numeric_dtype(result_df[col]) and col not in exclude_cols:
            keep_cols.append(col)
        # Keep specified text columns
        elif col in text_cols:
            keep_cols.append(col)
    
    result_df = result_df[keep_cols]
    
    # Ensure no missing values
    result_df = result_df.fillna(0)
    
    return result_df

def create_word2vec_embeddings(text_series, vector_size=100, min_count=2):
    """
    Create Word2Vec embeddings for text data
    
    Args:
        text_series (pd.Series): Series containing preprocessed text
        vector_size (int): Size of the word vectors
        min_count (int): Minimum frequency of words to include
        
    Returns:
        dict: Dictionary mapping documents to vectors
        gensim.models.Word2Vec: Trained Word2Vec model
    """
    # Ensure we have valid text data
    processed_texts = text_series.apply(lambda x: preprocess_text(str(x)))
    
    # Train Word2Vec model
    w2v_model = Word2Vec(
        sentences=processed_texts,
        vector_size=vector_size,
        min_count=min_count,
        workers=4,
        window=5,
        sg=1  # Skip-gram model
    )
    
    print(f"Trained Word2Vec model with {len(w2v_model.wv)} words in vocabulary")
    
    # Create document vectors by averaging word vectors
    doc_vectors = {}
    for i, doc in enumerate(processed_texts):
        if len(doc) > 0:
            # Get vectors for all words in vocabulary
            word_vectors = [w2v_model.wv[word] for word in doc if word in w2v_model.wv]
            
            if len(word_vectors) > 0:
                # Average the word vectors
                doc_vectors[i] = np.mean(word_vectors, axis=0)
            else:
                # Use a zero vector if no words in vocabulary
                doc_vectors[i] = np.zeros(vector_size)
        else:
            # Use a zero vector for empty documents
            doc_vectors[i] = np.zeros(vector_size)
    
    return doc_vectors, w2v_model

def create_bert_embeddings(text_series, model_name='bert-base-uncased'):
    """
    Create BERT embeddings for text data
    
    Args:
        text_series (pd.Series): Series containing text
        model_name (str): Name of the BERT model to use
        
    Returns:
        dict: Dictionary mapping documents to vectors
    """
    # Load BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # Put model in evaluation mode
    model.eval()
    
    # Create document vectors
    doc_vectors = {}
    
    # Process in batches to avoid memory issues
    batch_size = 16
    
    for i in range(0, len(text_series), batch_size):
        batch_texts = text_series.iloc[i:i+batch_size].tolist()
        
        # Tokenize batch
        encoded_batch = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        
        # Get BERT embeddings
        with torch.no_grad():
            outputs = model(**encoded_batch)
            
            # Use CLS token embedding as document representation
            for j, embedding in enumerate(outputs.last_hidden_state[:, 0, :]):
                doc_vectors[i + j] = embedding.numpy()
    
    print(f"Created BERT embeddings for {len(doc_vectors)} documents")
    
    return doc_vectors

def create_combined_embeddings(df, text_col=None, embedding_method='word2vec'):
    """
    Create combined embeddings from numerical features and text
    
    Args:
        df (pd.DataFrame): DataFrame with features
        text_col (str): Column containing text data
        embedding_method (str): Method to use for text embedding ('word2vec' or 'bert')
        
    Returns:
        numpy.ndarray: Combined embeddings
        dict: Additional information about the embeddings
    """
    # Prepare data
    numeric_df = df.select_dtypes(include=np.number)
    
    # Initialize embeddings with numerical data
    numeric_data = numeric_df.values
    
    # Create text embeddings if requested
    text_vectors = None
    text_model = None
    
    if text_col is not None and text_col in df.columns:
        print(f"Creating text embeddings using {embedding_method}")
        
        if embedding_method.lower() == 'word2vec':
            text_vectors, text_model = create_word2vec_embeddings(df[text_col])
            
            # Convert to array format
            text_array = np.zeros((len(df), next(iter(text_vectors.values())).shape[0]))
            for idx, vector in text_vectors.items():
                if idx < len(text_array):
                    text_array[idx] = vector
                    
        elif embedding_method.lower() == 'bert':
            text_vectors = create_bert_embeddings(df[text_col])
            
            # Convert to array format
            text_array = np.zeros((len(df), next(iter(text_vectors.values())).shape[0]))
            for idx, vector in text_vectors.items():
                if idx < len(text_array):
                    text_array[idx] = vector
        
        # Combine numeric and text data
        combined_data = np.hstack((numeric_data, text_array))
        print(f"Combined embeddings shape: {combined_data.shape}")
    else:
        # Use only numerical data
        combined_data = numeric_data
        print(f"Using only numerical features. Embedding shape: {combined_data.shape}")
    
    return combined_data, {
        'numeric_columns': numeric_df.columns.tolist(),
        'text_model': text_model,
        'embedding_method': embedding_method
    }

def reduce_dimensions(embeddings, method='tsne', n_components=2):
    """
    Reduce the dimensionality of embeddings for visualization
    
    Args:
        embeddings (numpy.ndarray): High-dimensional embeddings
        method (str): Method to use ('tsne' or 'umap')
        n_components (int): Number of components for the low-dimensional space
        
    Returns:
        numpy.ndarray: Low-dimensional embeddings
    """
    if method.lower() == 'tsne':
        # t-SNE reduction
        tsne = TSNE(
            n_components=n_components,
            perplexity=min(30, len(embeddings) - 1),
            n_iter=1000,
            random_state=42
        )
        reduced_embeddings = tsne.fit_transform(embeddings)
        print(f"Reduced dimensions using t-SNE to {n_components} components")
        
    elif method.lower() == 'umap':
        # UMAP reduction
        reducer = UMAP(
            n_components=n_components,
            n_neighbors=min(15, len(embeddings) - 1),
            min_dist=0.1,
            random_state=42
        )
        reduced_embeddings = reducer.fit_transform(embeddings)
        print(f"Reduced dimensions using UMAP to {n_components} components")
        
    else:
        raise ValueError(f"Unknown dimensionality reduction method: {method}")
    
    return reduced_embeddings

def evaluate_embedding_quality(embeddings, labels=None):
    """
    Evaluate the quality of embeddings
    
    Args:
        embeddings (numpy.ndarray): Embeddings to evaluate
        labels (numpy.ndarray): Optional labels for supervised evaluation
        
    Returns:
        dict: Evaluation metrics
    """
    metrics = {}
    
    # Clustering-based evaluation
    k_values = [2, 3, 5, 8]
    silhouette_scores = []
    
    for k in k_values:
        if k < len(embeddings):
            # Cluster the embeddings
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(embeddings)
            
            # Calculate silhouette score
            if len(np.unique(cluster_labels)) > 1:  # Need at least 2 clusters
                score = silhouette_score(embeddings, cluster_labels)
                silhouette_scores.append((k, score))
    
    if silhouette_scores:
        metrics['silhouette_scores'] = silhouette_scores
        best_k, best_score = max(silhouette_scores, key=lambda x: x[1])
        metrics['best_k'] = best_k
        metrics['best_silhouette'] = best_score
    
    # Calculate embedding variance
    embedding_variance = np.var(embeddings, axis=0).mean()
    metrics['embedding_variance'] = embedding_variance
    
    # Calculate pairwise distances
    sample_size = min(1000, len(embeddings))
    indices = np.random.choice(len(embeddings), sample_size, replace=False)
    sampled_embeddings = embeddings[indices]
    
    pairwise_distances = []
    for i in range(sample_size):
        for j in range(i + 1, min(i + 11, sample_size)):
            dist = np.linalg.norm(sampled_embeddings[i] - sampled_embeddings[j])
            pairwise_distances.append(dist)
    
    metrics['avg_pairwise_distance'] = np.mean(pairwise_distances)
    metrics['std_pairwise_distance'] = np.std(pairwise_distances)
    
    return metrics

# Visualizing Embeddings

In [16]:
def create_and_visualize_embeddings(dataset_name, df, text_col=None, color_col=None, methods=['word2vec', 'bert']):
    """
    Create embeddings for a dataset and visualize them
    
    Args:
        dataset_name (str): Name of the dataset
        df (pd.DataFrame): DataFrame with features
        text_col (str): Column containing text data
        color_col (str): Column to use for coloring points
        methods (list): List of embedding methods to use
        
    Returns:
        dict: Dictionary with embeddings and evaluation metrics
    """
    results = {}
    
    for method in methods:
        print(f"\nCreating {method} embeddings for {dataset_name} dataset")
        
        # Create combined embeddings
        embeddings, embedding_info = create_combined_embeddings(df, text_col, method)
        
        # Evaluate embedding quality
        evaluation_metrics = evaluate_embedding_quality(embeddings)
        
        print(f"Embedding evaluation for {method}:")
        for metric, value in evaluation_metrics.items():
            if isinstance(value, list):
                print(f"  {metric}: {[f'k={k}, score={s:.4f}' for k, s in value]}")
            else:
                print(f"  {metric}: {value:.4f}")
        
        # Reduce dimensions for visualization
        tsne_embeddings = reduce_dimensions(embeddings, 'tsne', 2)
        umap_embeddings = reduce_dimensions(embeddings, 'umap', 2)
        
        # Store results
        results[method] = {
            'embeddings': embeddings,
            'tsne_embeddings': tsne_embeddings,
            'umap_embeddings': umap_embeddings,
            'metrics': evaluation_metrics,
            'info': embedding_info
        }
        
        # Visualize reduced embeddings
        for dim_method, reduced_embeddings in [('t-SNE', tsne_embeddings), ('UMAP', umap_embeddings)]:
            plt.figure(figsize=(10, 8))
            
            if color_col and color_col in df.columns:
                # Get values for coloring
                if pd.api.types.is_numeric_dtype(df[color_col]):
                    # Use a colormap for numeric values
                    scatter = plt.scatter(
                        reduced_embeddings[:, 0], 
                        reduced_embeddings[:, 1],
                        c=df[color_col],
                        cmap='viridis',
                        alpha=0.7,
                        s=30
                    )
                    plt.colorbar(scatter, label=color_col)
                else:
                    # Use categorical colors for non-numeric values
                    categories = df[color_col].astype('category').cat.categories
                    colors = df[color_col].astype('category').cat.codes
                    scatter = plt.scatter(
                        reduced_embeddings[:, 0], 
                        reduced_embeddings[:, 1],
                        c=colors,
                        cmap='tab10',
                        alpha=0.7,
                        s=30
                    )
                    plt.legend(handles=scatter.legend_elements()[0], labels=categories, title=color_col)
            else:
                # No coloring
                plt.scatter(
                    reduced_embeddings[:, 0], 
                    reduced_embeddings[:, 1],
                    alpha=0.7,
                    s=30
                )
            
            plt.title(f"{dataset_name} - {method} Embeddings - {dim_method}")
            plt.xlabel(f"{dim_method} Dimension 1")
            plt.ylabel(f"{dim_method} Dimension 2")
            plt.tight_layout()
            
            # Save the visualization
            filename = f"visualizations/{dataset_name.lower()}_{method}_{dim_method.lower().replace('-', '')}.png"
            plt.savefig(filename, dpi=300)
            plt.close()
            print(f"Saved visualization to {filename}")
    
    return results

# Create embeddings for each dataset
embedding_results = {}

# Flood dataset
embedding_results['Floods'] = create_and_visualize_embeddings(
    'Floods',
    fe_flood_df,
    text_col='location' if 'location' in fe_flood_df.columns else None,
    color_col='location' if 'location' in fe_flood_df.columns else None,
    methods=['word2vec']  # Use only Word2Vec for faster execution
)

# Earthquake dataset
embedding_results['Earthquakes'] = create_and_visualize_embeddings(
    'Earthquakes',
    fe_earthquake_df,
    text_col=None,
    color_col='magnitude_category' if 'magnitude_category' in fe_earthquake_df.columns else 'magnitude',
    methods=['word2vec']
)

# Landslide dataset
trigger_col = next((col for col in fe_landslide_df.columns if 'trigger' in col.lower()), None)
embedding_results['Landslides'] = create_and_visualize_embeddings(
    'Landslides',
    fe_landslide_df,
    text_col=trigger_col,
    color_col=trigger_col,
    methods=['word2vec']
)

# Shelter dataset
embedding_results['Shelters'] = create_and_visualize_embeddings(
    'Shelters',
    fe_shelter_df,
    text_col='name' if 'name' in fe_shelter_df.columns else None,
    color_col='type' if 'type' in fe_shelter_df.columns else 'country',
    methods=['word2vec']
)

# Save embedding models and reduced embeddings
os.makedirs('models/embeddings', exist_ok=True)

for dataset_name, result in embedding_results.items():
    for method, data in result.items():
        # Save reduced embeddings
        np.save(f'models/embeddings/{dataset_name.lower()}_{method}_tsne.npy', data['tsne_embeddings'])
        np.save(f'models/embeddings/{dataset_name.lower()}_{method}_umap.npy', data['umap_embeddings'])
        
        # Save full embeddings
        np.save(f'models/embeddings/{dataset_name.lower()}_{method}_full.npy', data['embeddings'])
        
        # Save Word2Vec model if available
        if method == 'word2vec' and data['info']['text_model'] is not None:
            data['info']['text_model'].save(f'models/embeddings/{dataset_name.lower()}_{method}_model')

# Create a summary of embedding results
embedding_summary = []
for dataset_name, result in embedding_results.items():
    for method, data in result.items():
        embedding_summary.append({
            'Dataset': dataset_name,
            'Method': method,
            'Embedding Dimensions': data['embeddings'].shape[1],
            'Best Silhouette Score': data['metrics'].get('best_silhouette', 'N/A'),
            'Best K': data['metrics'].get('best_k', 'N/A'),
            'Avg Pairwise Distance': data['metrics'].get('avg_pairwise_distance', 'N/A'),
            'Embedding Variance': data['metrics'].get('embedding_variance', 'N/A')
        })

# Display embedding summary
embedding_summary_df = pd.DataFrame(embedding_summary)
print("\nEmbedding Summary:")
print(embedding_summary_df)

# Save embedding summary
embedding_summary_df.to_csv('reports/embedding_summary.csv', index=False)

# Create comparison visualization of embedding methods
datasets = embedding_summary_df['Dataset'].unique()

# Compare silhouette scores across methods
plt.figure(figsize=(12, 6))
for method in embedding_summary_df['Method'].unique():
    method_data = embedding_summary_df[embedding_summary_df['Method'] == method]
    
    # Ensure we have data for each dataset with this method
    dataset_scores = []
    method_datasets = []
    for dataset in datasets:
        dataset_row = method_data[method_data['Dataset'] == dataset]
        if len(dataset_row) > 0 and dataset_row['Best Silhouette Score'].iloc[0] != 'N/A':
            dataset_scores.append(float(dataset_row['Best Silhouette Score'].iloc[0]))
            method_datasets.append(dataset)
    
    if dataset_scores:
        plt.plot(method_datasets, dataset_scores, marker='o', label=method)

plt.xlabel('Dataset')
plt.ylabel('Best Silhouette Score')
plt.title('Embedding Quality Comparison (Higher is Better)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('visualizations/embedding_quality_comparison.png', dpi=300)
plt.close()


Creating word2vec embeddings for Floods dataset
Using only numerical features. Embedding shape: (50000, 28)
Embedding evaluation for word2vec:
  silhouette_scores: ['k=2, score=0.3202', 'k=3, score=0.3300', 'k=5, score=0.3205', 'k=8, score=0.2663']
  best_k: 3.0000
  best_silhouette: 0.3300
  embedding_variance: 1.4461
  avg_pairwise_distance: 8.2826
  std_pairwise_distance: 3.5121
Reduced dimensions using t-SNE to 2 components
Reduced dimensions using UMAP to 2 components
Saved visualization to visualizations/floods_word2vec_tsne.png
Saved visualization to visualizations/floods_word2vec_umap.png

Creating word2vec embeddings for Earthquakes dataset
Using only numerical features. Embedding shape: (51275, 46)


ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values